In [3]:
#Step 1: Establish a connection between Python and the Sakila database.
!pip install pandas sqlalchemy mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable


In [12]:
##Step 3: Python function to count rentals made by each customer.
def rental_count_month(df, month, year):
    # Ensure the rental_date column is in datetime format
    df['rental_date'] = pd.to_datetime(df['rental_date'])
    df_filtered = df[(df['rental_date'].dt.month == month) & (df['rental_date'].dt.year == year)]
    rental_count = df_filtered.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_count

In [13]:
##Step 4: Python function to compare rentals between two months and years.
def compare_rentals(df1, df2):
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    month1_col = df1.columns[1]
    month2_col = df2.columns[1]
    combined_df['difference'] = combined_df[month1_col] - combined_df[month2_col]
    return combined_df

In [6]:
import pandas as pd
from sqlalchemy import create_engine
import mysql.connector
from mysql.connector import Error
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
#import pymysql
import getpass

# Replace with your MySQL credentials
username = 'root'
host = 'localhost'
database = 'sakila'



password = getpass.getpass("Please enter the SQL database password: ")

# Create a database engine
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}/{database}')

In [7]:
def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            database=db_name
        )
        if connection.is_connected():
            print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

In [8]:
# s
connection = create_connection(host, username, password, "fraud_detection")

Connection to MySQL DB successful


In [11]:
##Step 2: Python function to retrieve rental data for a given month and year
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

def rentals_month(engine, month, year):
    query = text("""
    SELECT rental_id, customer_id, rental_date
    FROM rental
    WHERE MONTH(rental_date) = :month AND YEAR(rental_date) = :year
    """)
    with engine.connect() as connection:
        result = connection.execute(query, month=month, year=year).fetchall()
        df = pd.DataFrame(result, columns=['rental_id', 'customer_id', 'rental_date'])
    return df

# Retrieve rental data for May and June 2005
rentals_may = rentals_month(engine, 5, 2005)
rentals_june = rentals_month(engine, 6, 2005)

In [10]:
# Count rentals for each customer in May and June 2005
rental_count_may = rental_count_month(rentals_may, 5, 2005)
rental_count_june = rental_count_month(rentals_june, 6, 2005)

NameError: name 'rentals_may' is not defined

In [ ]:
def compare_rentals(df1, df2):
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    month1_col = df1.columns[1]
    month2_col = df2.columns[1]
    combined_df['difference'] = combined_df[month1_col] - combined_df[month2_col]
    return combined_df

In [ ]:
# Compare the rental counts between May and June 2005
comparison_df = compare_rentals(rental_count_may, rental_count_june)

# Display the comparison DataFrame
print(comparison_df)

     customer_id  rentals_05_2005  rentals_06_2005  difference
0              1              2.0              7.0        -5.0
1              2              1.0              1.0         0.0
2              3              2.0              4.0        -2.0
3              5              3.0              5.0        -2.0
4              6              3.0              4.0        -1.0
..           ...              ...              ...         ...
593          583              0.0              6.0        -6.0
594          585              0.0              4.0        -4.0
595          591              0.0              3.0        -3.0
596          592              0.0              5.0        -5.0
597          598              0.0              1.0        -1.0

[598 rows x 4 columns]
